In [3]:
# """网易云音乐下载程序"""


import re
import requests
from selenium import webdriver


class DownloadMusic(object):
    def __init__(self):
        self.music_api = 'http://music.163.com/song/media/outer/url?id={}.mp3'
        self.music_search = 'https://music.163.com/#/search/m/?s={}'
        # Mozilla是一家非盈利公司
        self.headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        self.browser = webdriver.Chrome(options=chrome_options)

    def __del__(self):
        """
        运行结束自动关闭webdriver
        :return:
        """
        self.browser.close()

    def search_music(self, param):
        """
        传入搜索关键字，并请求网页
        :param param: 搜索关键字
        :return:
        """
        url = self.music_search.format(param)
        self.browser.get(url)
        # 值得注意的是这里需要跳转iframe，不懂请自行百度
        self.browser.switch_to_frame("g_iframe") #use driver.switch_to.frame instead
        html = self.browser.page_source
        self.parse_html(html)

    def parse_html(self, html):
        """
        传入html，解析网页
        :param html: 
        :return:
        """
        song_id = re.findall('a href="/song\?id=(.*?)"', html)
        song_name = re.findall(('<b title="(.*?)"'), html)
        singer = re.findall('span class="s-fc7">(.*?)</span>', html)
        self.download_music(song_id, song_name, singer)

    def download_music(self, song_id, song_name, singer):
        """
        传入歌曲id，歌曲名，歌手并下载歌曲
        :param song_id: 
        :param song_name: 
        :param singer: 
        :return:
        """
        for id, name, singer in zip(song_id, song_name, singer):
            song_url = self.music_api.format(id)
            resp = requests.get(song_url)
            with open('%s-%s.mp3' % (singer, name), 'wb') as f:
                f.write(resp.content)
                print('已下载 歌曲：%s-%s.mp3' % (singer, name))
                f.close()

    def main(self, param):
        self.search_music(param)


if __name__ == '__main__':
    spider = DownloadMusic()
    param = input("请输入歌手或歌名：")
    spider.main(param)

请输入歌手或歌名： yellow


e:\python\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: use driver.switch_to.frame instead


已下载 歌曲：Yellow-Yellow.mp3
已下载 歌曲：Yellow-Yellow（翻自 有機酸） .mp3
已下载 歌曲：yellow-YELLOW.mp3
已下载 歌曲：YELLOW-Yellow.mp3
已下载 歌曲：Yellow-YELLOW（翻自 神山羊）.mp3
已下载 歌曲：Yellow-yellow.mp3


FileNotFoundError: [Errno 2] No such file or directory: 'YELLOW-有機酸-YELLOW（多人合唱×混响）（翻自 神山羊）（Reigo_叉叉 / 困先生 / 阿两 / 俺酱 remix） .mp3'